In [1]:
!source ~/venv-ptmetal/bin/activate

In [2]:
!pip show torchvision && pip install torchvision --upgrade

Name: torchvision
Version: 0.16.1
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /Users/davidledbetter/miniconda/envs/tf/lib/python3.9/site-packages
Requires: numpy, pillow, requests, torch
Required-by: 
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 12.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.1.1
    Uninstalling torch-2.1.1:
      Successfully uninstalled torch-2.1.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.1
    Uninstalling torchvision-0.16.1:
      Successfully uninstalled torchvision-0.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [3]:
import torch
from torch import nn
import torch.nn.functional as F

device = "cpu"
if torch.backends.mps.is_available():
    device = "mps"
    print("Using MPS device for GPU acceleration!")
else:
    print("MPS device not found. Falling back to CPU.")

Using MPS device for GPU acceleration!


In [4]:
import time

start_time = time.time()

torch.mps.synchronize()
a = torch.ones(4000,4000).to(device)
for _ in range(500):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.29730987548828125


In [5]:
!curl -o model.h5 https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [6]:
# !wget -O model.h5 https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [7]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [8]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  2.1 ; cuda:  2.1.2
detectron2: 0.6


In [9]:
import torch
from torch import nn
from torch.fft import fft2, ifft2
from torch.autograd import Variable
import cv2
from torch.nn import DataParallel
from torchvision.models import resnet50
from transformers import AutoModelForSequenceClassification

device = "cpu"
if torch.backends.mps.is_available():
    device = "mps"
    print("Using MPS device for GPU acceleration!")
else:
    print("MPS device not found. Falling back to CPU.")

def farneback_mps(I1, I2, pyr_levels, eps, tol):
    Ix1, Iy1 = _compute_derivatives(I1)
    Ix2, Iy2 = _compute_derivatives(I2)
    flow = torch.zeros_like(Ix1)
    for level in range(pyr_levels):
        warped_I2 = _warp_image(I2, flow * 2 ** level)
        diff = warped_I2 - I1
        Ixx, Iyy = _compute_derivatives(diff)
        error = torch.sqrt(Ixx**2 + Iyy**2)
        flow = _update_flow(flow, Ix1, Iy1, Ixx, Iyy, error, eps, tol)
    flow = flow / 2 ** (pyr_levels - 1)
    return flow

def _compute_derivatives(image):
    sobel_x = torch.tensor([-1, 0, 1, -2, 0, 2, -1, 0, 1]).view(3, 3).to("mps")
    sobel_y = torch.tensor([1, 2, 1, 0, 0, 0, -1, -2, -1]).view(3, 3).to("mps")
    Ix = F.conv2d(image, sobel_x, padding=1, groups=3).to("mps")
    Iy = F.conv2d(image, sobel_y, padding=1, groups=3).to("mps")
    return Ix, Iy

def _warp_image(image, flow):
    flow_x, flow_y = flow.split(2, dim=1)
    grid_x, grid_y = F.affine_grid(flow, image.shape, align_corners=False)
    warped_image = F.grid_sample(image, torch.stack([grid_x, grid_y], dim=3), mode="bilinear", padding_mode="border", align_corners=False).to("mps")
    return warped_image

def _update_flow(flow, Ix1, Iy1, Ixx, Iyy, error, eps, tol):
    flow_dx = F.conv2d(flow, Ixx, padding=1).to("mps")
    flow_dy = F.conv2d(flow, Iyy, padding=1).to("mps")
    flow -= (Ix1 * flow_dx + Iy1 * flow_dy) / (error**2 + eps)
    smooth_flow = F.conv2d(flow, torch.ones((3, 3)).to("mps"), padding=1).to("mps")
    flow = (1 - tol) * flow + tol * smooth_flow
    return flow

def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (800, 600))
    normalized_frame = resized_frame / 255.0
    rgb_frame = cv2.cvtColor(normalized_frame, cv2.COLOR_BGR2RGB)
    frame_tensor = torch.from_numpy(rgb_frame).permute(2, 0, 1).to(device)
    return frame_tensor

model_tf = load_model("model.h5")
model = nn.Sequential(*[layer.to(nn.device) for layer in model_tf.layers])
for name, param in model_tf.get_layer("mrcnn_mask_head_fc").items():
    model_torch.state_dict()[name].copy_(param)
torch.save(model.state_dict(), "model.pth")
print("installing")
import requests
print("Model loaded and moved to device.")

def calculate_segment_velocity(segment, prev_frame, current_frame):
    mask = masks[segment]
    masked_prev_frame = prev_frame * mask
    masked_current_frame = current_frame * mask
    segment_flow = farneback_mps(masked_prev_frame, masked_current_frame, pyr_levels=3, eps=0.01, tol=0.5)
    segment_velocity = torch.sum(segment_flow) / torch.sum(mask)
    return segment_velocity.cpu().numpy()

def visualize_results(frame, segments, masks, velocities):
    for segment, mask, bbox, velocity in zip(segments, masks, segment_boxes, velocities):
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 2)
        start_point = (int(bbox[0] + bbox[2]/2), int(bbox[1] + bbox[3]/2))
        end_point = (start_point[0] + int(velocity[0]), start_point[1] + int(velocity[1]))
        cv2.arrowedLine(frame, start_point, end_point, (0, 0, 255), 2)
    cv2.imshow("Video", frame)

cap = cv2.VideoCapture("video.mp4")
ret, prev_frame = cap.read()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    prev_frame = preprocess_frame(prev_frame)
    current_frame = preprocess_frame(frame)

    with torch.no_grad():
        predictions = model(current_frame.unsqueeze(0))
        segments = predictions[0].cpu().numpy()
        masks = predictions[1].cpu().numpy()

    velocities = []
    for segment in segments:
        segment_velocity = calculate_segment_velocity(segment, prev_frame, current_frame)
        velocities.append(segment_velocity)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Initializing MLIR with module: _site_initialize_0
Registering dialects from initializer <module 'jaxlib.mlir._mlir_libs._site_initialize_0' from '/Users/davidledbetter/miniconda/envs/tf/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs/_site_initialize_0.so'>
etils.epath found. Using etils.epath for file I/O.
Using MPS device for GPU acceleration!


NameError: name 'load_model' is not defined

# Optical Flow Only

In [ ]:
import numpy as np
import cv2

def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (800, 600))
    normalized_frame = resized_frame / 255.0
    rgb_frame = cv2.normalize(normalized_frame, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    rgb_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2RGB)
    return rgb_frame

def draw_optical_flow_hsv(flow):
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 255
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

def draw_optical_flow(frame, flow):
    h, w = frame.shape[:2]
    step = 16
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    vis = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    for (x, y), (fx, fy) in zip(np.stack([x, y], axis=-1), np.stack([fx, fy], axis=-1)):
        cv2.line(vis, (x, y), (x+int(fx), y+int(fy)), (0, 255, 0), 1)
        cv2.circle(vis, (x, y), 1, (0, 255, 0), -1)
    return vis

def stack_frames(original_frame, processed_frame):
    processed_resized = cv2.resize(processed_frame, (original_frame.shape[1], original_frame.shape[0]))
    stacked_frame = np.hstack((original_frame, processed_resized))
    return stacked_frame

cap = cv2.VideoCapture("video.mp4")

ret, frame = cap.read()

if not ret:
    print("Error: Could not open video file")
    exit()

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

frames_for_gif = []

i = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if (i % 10 == 0):
        print(i, "frames completed")
    i += 1

    frame = preprocess_frame(frame)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.resize(prev_gray, (gray.shape[1], gray.shape[0]))

    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    flow_hsv = draw_optical_flow_hsv(flow)

    stacked = stack_frames(frame, flow_hsv)
    stacked = cv2.cvtColor(stacked, cv2.COLOR_BGR2RGB)
    frames_for_gif.append(stacked)

    prev_gray = gray

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0 frames completed
10 frames completed
20 frames completed
30 frames completed
40 frames completed
50 frames completed
60 frames completed
70 frames completed
80 frames completed
90 frames completed
100 frames completed
110 frames completed
120 frames completed
130 frames completed
140 frames completed
150 frames completed
160 frames completed
170 frames completed
180 frames completed
190 frames completed
200 frames completed
210 frames completed
220 frames completed
230 frames completed
240 frames completed
250 frames completed


In [ ]:
import imageio
imageio.mimsave('output.gif', frames_for_gif, duration=0.1)